In [1]:
pip install geopandas

     |████████████████████████████████| 972kB 5.9MB/s 
     |████████████████████████████████| 14.8MB 313kB/s 
     |████████████████████████████████| 6.5MB 1.8MB/s 


In [2]:
import pandas as pd
import numpy as np
from google.colab import files
import geopandas as gpd
import traceback

In [3]:
!wget -O WesterAustraliaLGAPopulation.gpkg "https://drive.google.com/uc?id=1XzcmhZOtVvlcCJ5WaVNoAAAdWthU-zmq"

--2021-01-22 22:24:11--  https://drive.google.com/uc?id=1XzcmhZOtVvlcCJ5WaVNoAAAdWthU-zmq
Resolving drive.google.com (drive.google.com)... 108.177.112.138, 108.177.112.139, 108.177.112.113, ...
Connecting to drive.google.com (drive.google.com)|108.177.112.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0o-8o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/o37j488o6218q917bii0qrrle9lalvjk/1611354225000/16288098520104315905/*/1XzcmhZOtVvlcCJ5WaVNoAAAdWthU-zmq [following]
--2021-01-22 22:24:13--  https://doc-0o-8o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/o37j488o6218q917bii0qrrle9lalvjk/1611354225000/16288098520104315905/*/1XzcmhZOtVvlcCJ5WaVNoAAAdWthU-zmq
Resolving doc-0o-8o-docs.googleusercontent.com (doc-0o-8o-docs.googleusercontent.com)... 142.250.125.132, 2607:f8b0:4001:c2f::84
Connecting to doc-0o-8o-docs.googleusercontent.com (doc-0o-8o-docs.googleusercontent.com)|1

In [4]:
data = gpd.read_file("WesterAustraliaLGAPopulation.gpkg")

/usr/local/lib/python3.6/dist-packages/geopandas/geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [5]:
wadata=data[data['State_name_2019']=='Western Australia'].reset_index()
wadata['ERP_2020']=wadata['ERP_2019'] # We assume population in 2020 equal to 2019 as data not available

In [6]:
wadata.head()

,index,State_code_2019,State_name_2019,LGA_code_2019,LGA_name_2019,ERP_2001,ERP_2002,ERP_2003,ERP_2004,ERP_2005,ERP_2006,ERP_2007,ERP_2008,ERP_2009,ERP_2010,ERP_2011,ERP_2012,ERP_2013,ERP_2014,ERP_2015,ERP_2016,ERP_2017,ERP_2018,ERP_2019,ERP_change_number_2018_19,ERP_change_per_cent_2018_19,Area_km2,Pop_density_2019_people_per_km2,Births_2016_17,Deaths_2016_17,Natural_increase_2016_17,Internal_arrivals_2016_17,Internal_departures_2016_17,Net_internal_migration_2016_17,Overseas_arrivals_2016_17,Overseas_departures_2016_17,Net_overseas_migration_2016_17,Births_2017_18,Deaths_2017_18,Natural_increase_2017_18,Internal_arrivals_2017_18,Internal_departures_2017_18,Net_internal_migration_2017_18,Overseas_arrivals_2017_18,Overseas_departures_2017_18,Net_overseas_migration_2017_18,Births_2018_19,Deaths_2018_19,Natural_increase_2018_19,Internal_arrivals_2018_19,Internal_departures_2018_19,Net_internal_migration_2018_19,Overseas_arrivals_2018_19,Overseas_departures_2018_19,Net_overseas_migration_2018_19,geometry,ERP_2020
0,358,5,Western Australia,50080,Albany (C),31362,31713,32020,32237,32726,33171,33310,33704,34164,34518,34873,35643,36318,36919,37206,37285,37555,37820,38053,233,0.6161,4310.890625,8.827200,387.0,333.0,54.0,2096.0,1956.0,140.0,275.0,199.0,76.0,421.0,312.0,109.0,2244.0,2163.0,81.0,250.0,175.0,75.0,360.0,332.0,28.0,2320.0,2214.0,106.0,260.0,161.0,99.0,"MULTIPOLYGON (((117.95914 -35.09446, 117.95910...",38053
1,359,5,Western Australia,50210,Armadale (C),52391,52204,52049,52041,51795,52299,53735,55809,58508,61858,65395,69101,72662,75559,78848,82021,84669,87607,90797,3190,3.6413,559.526123,162.274796,1540.0,379.0,1161.0,7526.0,6350.0,1176.0,1145.0,834.0,311.0,1546.0,410.0,1136.0,8429.0,6935.0,1494.0,1039.0,731.0,308.0,1569.0,400.0,1169.0,8700.0,7084.0,1616.0,1080.0,675.0,405.0,"POLYGON ((116.18955 -32.28743, 116.12649 -32.2...",90797
2,360,5,Western Australia,50250,Ashburton (S),5969,6138,6467,6925,7263,7608,7939,8480,9004,9510,10227,11194,12025,12515,13002,13311,13273,13327,13305,-22,-0.1651,100818.210938,0.132000,104.0,9.0,95.0,787.0,956.0,-169.0,135.0,99.0,36.0,95.0,12.0,83.0,989.0,1054.0,-65.0,122.0,86.0,36.0,101.0,16.0,85.0,914.0,1069.0,-155.0,127.0,79.0,48.0,"MULTIPOLYGON (((115.57891 -20.67466, 115.57891...",13305
3,361,5,Western Australia,50280,Augusta-Margaret River (S),10228,10346,10639,10791,10936,11052,11195,11522,11766,11951,12228,12870,13487,13996,14419,14819,15234,15694,16172,478,3.0457,2122.375000,7.619800,188.0,81.0,107.0,986.0,752.0,234.0,271.0,197.0,74.0,196.0,84.0,112.0,1134.0,858.0,276.0,245.0,173.0,72.0,191.0,58.0,133.0,1208.0,957.0,251.0,254.0,160.0,94.0,"MULTIPOLYGON (((114.99859 -34.04181, 114.99849...",16172
4,362,5,Western Australia,50350,Bassendean (T),14055,14165,14030,13996,13943,13896,14099,14358,14626,14897,15179,15527,15828,16003,15842,15613,15617,15735,15823,88,0.5593,10.344900,1529.546021,199.0,107.0,92.0,1428.0,1559.0,-131.0,162.0,119.0,43.0,237.0,123.0,114.0,1628.0,1669.0,-41.0,148.0,103.0,45.0,213.0,120.0,93.0,1722.0,1784.0,-62.0,153.0,96.0,57.0,"POLYGON ((115.95952 -31.89435, 115.95888 -31.8...",15823


In [7]:
#Get Data set ver5
df = pd.read_excel('https://drive.google.com/uc?id=171VNl8ZUNLNNB1-dCyMXb7zyzzsjoLk1', header=0)

In [8]:
#Make a geometry column as a lat lon point
df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.LON, df.LAT, crs='EPSG:4283'))

In [9]:
df['population']=np.nan
df['LGA']=None

In [10]:
df.head()

,fih_fire_s,fih_year1,fih_season,fih_distri,fih_hist_d,fih_number,fih_fire_t,fih_date1,fih_cause,fih_ignit_,fih_detect,fih_capt_m,fih_author,fih_poly_t,fih_commen,fih_name,fih_perime,LAT,LON,ACRES,MODIS,NC_LAT,NC_LON,lfmc_mean,lfmc_stdv,quality_mask,ELEVATION,dis_latlon_n_nclatlon,climate_LAT,climate_LON,rain_fall_mm,max_temperature_c,min_temperature_c,vp_9am_hpa,vp_3pm_hpa,dis_latlon_n_climatelatlon,geometry,population,LGA
0,2000/2001,2001,SU,PHL,Mundaring,999,WF,2001-01-01,999,0,0,HIM,,Actual Burnt,,,9.653766,-31.3497,116.0366,850.9043,h27v12,-31.350992,116.036200,36.224998,18.932627,1.0,206,148.566183,-31.35,116.05,0.0,29.32,13.40,15.27,15.10,1272.918883,POINT (116.03660 -31.34970),NaN,None
1,2000/2001,2001,SU,PHL,Mundaring,999,WF,2001-01-01,999,0,0,HIM,,Actual Burnt,,,1.058290,-32.1413,116.1956,18.3947,h27v12,-32.142631,116.195939,61.924999,16.209669,1.0,266,151.398109,-32.15,116.20,0.0,29.39,14.58,14.62,14.69,1052.361097,POINT (116.19560 -32.14130),NaN,None
2,2000/2001,2001,SU,PHL,Mundaring,999,WF,2001-01-01,999,0,0,HIM,,Actual Burnt,,,0.157169,-32.0572,116.1932,0.4853,h27v12,-32.059300,116.193215,66.547501,13.480727,1.0,312,233.564838,-32.05,116.20,0.0,29.14,14.32,14.67,14.76,1025.507101,POINT (116.19320 -32.05720),NaN,None
3,2000/2001,2001,SU,PHL,Mundaring,999,WF,2001-01-01,999,0,0,HIM,,Actual Burnt,,,0.558105,-32.0542,116.1940,5.3271,h27v12,-32.055134,116.192840,61.877499,13.119061,1.0,311,150.803304,-32.05,116.20,0.0,29.14,14.32,14.67,14.76,733.392420,POINT (116.19400 -32.05420),NaN,None
4,2000/2001,2001,SU,PHL,Mundaring,999,WF,2001-01-01,999,0,0,HIM,,Actual Burnt,,,0.165064,-32.0476,116.1925,0.5353,h27v12,-32.046801,116.192091,83.957497,14.985291,1.0,309,96.865831,-32.05,116.20,0.0,29.14,14.32,14.67,14.76,755.563084,POINT (116.19250 -32.04760),NaN,None


In [11]:
for index, row in df.iterrows():
  if (index % 1000)==0:
    print("Completed : "+str(round((index*100/df.shape[0]),2))+"%")
  year=row['fih_date1'].strftime('%Y')
  for pindex, pop in wadata.iterrows():
    try:
      if row.geometry.within(pop.geometry): #use this https://automating-gis-processes.github.io/2016/Lesson3-point-in-polygon.html
        df.at[index, 'population']=pop['ERP_'+year]
        df.at[index, 'LGA']=pop['LGA_name_2019']
        break
    except Exception as e:
      print("===> "+pop['LGA_name_2019'])
      print("Population data :"+str(e))
      print(traceback.format_exc())
print("Completed : 100.0%")
#drop geometry column
df.drop('geometry', inplace=True, axis=1)

Completed : 0.0%
Completed : 2.85%
Completed : 5.69%
Completed : 8.54%
Completed : 11.38%
Completed : 14.23%
Completed : 17.08%
Completed : 19.92%
Completed : 22.77%
Completed : 25.61%
Completed : 28.46%
Completed : 31.31%
Completed : 34.15%
Completed : 37.0%
Completed : 39.84%
Completed : 42.69%
Completed : 45.54%
Completed : 48.38%
Completed : 51.23%
Completed : 54.07%
Completed : 56.92%
Completed : 59.77%
Completed : 62.61%
Completed : 65.46%
Completed : 68.3%
Completed : 71.15%
Completed : 74.0%
Completed : 76.84%
Completed : 79.69%
Completed : 82.53%
Completed : 85.38%
Completed : 88.23%
Completed : 91.07%
Completed : 93.92%
Completed : 96.76%
Completed : 99.61%
Completed : 100.0%


In [12]:
df.head()

,fih_fire_s,fih_year1,fih_season,fih_distri,fih_hist_d,fih_number,fih_fire_t,fih_date1,fih_cause,fih_ignit_,fih_detect,fih_capt_m,fih_author,fih_poly_t,fih_commen,fih_name,fih_perime,LAT,LON,ACRES,MODIS,NC_LAT,NC_LON,lfmc_mean,lfmc_stdv,quality_mask,ELEVATION,dis_latlon_n_nclatlon,climate_LAT,climate_LON,rain_fall_mm,max_temperature_c,min_temperature_c,vp_9am_hpa,vp_3pm_hpa,dis_latlon_n_climatelatlon,population,LGA
0,2000/2001,2001,SU,PHL,Mundaring,999,WF,2001-01-01,999,0,0,HIM,,Actual Burnt,,,9.653766,-31.3497,116.0366,850.9043,h27v12,-31.350992,116.036200,36.224998,18.932627,1.0,206,148.566183,-31.35,116.05,0.0,29.32,13.40,15.27,15.10,1272.918883,2948.0,Chittering (S)
1,2000/2001,2001,SU,PHL,Mundaring,999,WF,2001-01-01,999,0,0,HIM,,Actual Burnt,,,1.058290,-32.1413,116.1956,18.3947,h27v12,-32.142631,116.195939,61.924999,16.209669,1.0,266,151.398109,-32.15,116.20,0.0,29.39,14.58,14.62,14.69,1052.361097,52391.0,Armadale (C)
2,2000/2001,2001,SU,PHL,Mundaring,999,WF,2001-01-01,999,0,0,HIM,,Actual Burnt,,,0.157169,-32.0572,116.1932,0.4853,h27v12,-32.059300,116.193215,66.547501,13.480727,1.0,312,233.564838,-32.05,116.20,0.0,29.14,14.32,14.67,14.76,1025.507101,48743.0,Kalamunda (C)
3,2000/2001,2001,SU,PHL,Mundaring,999,WF,2001-01-01,999,0,0,HIM,,Actual Burnt,,,0.558105,-32.0542,116.1940,5.3271,h27v12,-32.055134,116.192840,61.877499,13.119061,1.0,311,150.803304,-32.05,116.20,0.0,29.14,14.32,14.67,14.76,733.392420,48743.0,Kalamunda (C)
4,2000/2001,2001,SU,PHL,Mundaring,999,WF,2001-01-01,999,0,0,HIM,,Actual Burnt,,,0.165064,-32.0476,116.1925,0.5353,h27v12,-32.046801,116.192091,83.957497,14.985291,1.0,309,96.865831,-32.05,116.20,0.0,29.14,14.32,14.67,14.76,755.563084,48743.0,Kalamunda (C)


In [13]:
df=df[(df['population'].notnull()) & (df['LGA'].notnull())]

In [14]:
df.describe()

,fih_year1,fih_cause,fih_detect,fih_perime,LAT,LON,ACRES,NC_LAT,NC_LON,lfmc_mean,lfmc_stdv,quality_mask,ELEVATION,dis_latlon_n_nclatlon,climate_LAT,climate_LON,rain_fall_mm,max_temperature_c,min_temperature_c,vp_9am_hpa,vp_3pm_hpa,dis_latlon_n_climatelatlon,population
count,34932.000000,34932.000000,34932.000000,3.493200e+04,34932.000000,34932.000000,3.493200e+04,34932.000000,34932.000000,34932.000000,34932.000000,27678.0,34932.000000,34932.000000,34932.000000,34932.000000,34932.000000,34932.000000,34932.000000,34932.000000,34932.000000,34932.000000,34932.000000
mean,2010.638240,509.727700,22.575490,6.782555e+02,-28.016166,119.483834,2.579403e+03,-28.016176,119.483825,73.325162,16.603412,1.0,285.587828,178.310006,-28.016270,119.483784,0.986228,30.246669,15.217987,12.603731,11.232864,1999.312163,16040.583734
std,5.930556,498.081921,135.945735,2.558101e+04,4.829776,3.841658,3.077619e+04,4.829784,3.841657,39.655383,5.814712,0.0,153.217771,66.051728,4.830031,3.841724,3.478098,8.666026,6.788165,5.070869,5.153552,751.719408,33131.146595
min,2001.000000,0.000000,0.000000,0.000000e+00,-35.112900,113.199200,0.000000e+00,-35.113361,113.197979,4.440000,3.458670,1.0,-17.000000,0.756998,-35.100000,113.200000,0.000000,10.960000,-1.460000,0.000000,0.000000,0.000000,79.000000
25%,2005.000000,1.000000,0.000000,6.595387e-01,-32.749700,116.097575,4.935200e+00,-32.750943,116.097096,55.580002,13.143747,1.0,173.000000,131.838467,-32.750000,116.100000,0.000000,23.360000,9.520000,9.310000,7.400000,1465.839440,1216.000000
50%,2012.000000,999.000000,0.000000,2.115346e+00,-28.243750,118.087800,2.716345e+01,-28.242767,118.087562,57.397499,15.137430,1.0,282.000000,185.208259,-28.250000,118.100000,0.000000,30.860000,14.570000,12.030000,11.050000,2076.718856,7667.000000
75%,2016.000000,999.000000,0.000000,1.075653e+01,-22.896375,122.603875,2.462207e+02,-22.897119,122.603640,91.173750,18.194876,1.0,406.000000,227.984598,-22.900000,122.600000,0.100000,37.430000,20.520000,14.980000,14.000000,2566.608375,12881.000000
max,2020.000000,999.000000,2875.000000,3.636410e+06,-20.094700,128.999900,3.756446e+06,-20.093050,129.001360,299.887512,76.268379,1.0,979.000000,476.192770,-20.100000,129.000000,64.900000,48.670000,33.290000,38.840000,35.700000,3784.391761,208237.000000


In [15]:
df.to_excel('WesternAustraliaFireData-ver6.xlsx',index=False)
files.download("/content/WesternAustraliaFireData-ver6.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>